In [58]:
import shap
import xgboost
from sklearn.model_selection import train_test_split
import matplotlib.pylab as pl
import pandas as pd
import numpy

In [59]:
data = pd.read_csv("AIDSdata.csv")
# We drop `id` because it's just the identification number of each patient
# We exclude `time_d` and `censor_d` because these seem unfair to include 
#     in modeling since they contain information about time and censor
# We chose not to include `txgrp` and `strat2` because they were derived 
# (and thus highly correlated with) from other predictor variables. 
data = data.drop(['id', 'time_d', 'censor_d', 'txgrp', 'strat2', 'time_d'], axis=1)

In [60]:
data.head(3)

,time,censor,tx,sex,raceth,ivdrug,hemophil,karnof,cd4,priorzdv,age
0,110,0,0,2,3,1,0,90,8.0,9.0,16
1,88,0,0,1,2,1,0,90,37.0,6.0,35
2,256,0,0,1,1,1,0,100,35.0,11.0,29


In [61]:
# We now separate predictors and labels
y = data[['time']]
X = data[['censor', 'tx', 'sex', 'raceth', 'ivdrug', 
          'hemophil', 'karnof', 'cd4', 'priorzdv', 'age']]

In [62]:
# create a train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [63]:
xgb_full = xgboost.DMatrix(X, label=y)
xgb_train = xgboost.DMatrix(X_train, label=y_train)
xgb_test = xgboost.DMatrix(X_test, label=y_test)

In [64]:
# use validation set to choose # of trees
params = {
    "eta": 0.002,
    "max_depth": 3,
    "objective": "survival:cox",
    "subsample": 0.5
}
model_train = xgboost.train(params, xgb_train, 10000, 
                            evals = [(xgb_test, "test")], verbose_eval=1000)

XGBoostError: b'[16:26:28] src/objective/objective.cc:21: Unknown objective function survival:cox\n\nStack trace returned 2 entries:\n[bt] (0) 0   libxgboost.dylib                    0x000000011b2277a9 _ZN4dmlc15LogMessageFatalD1Ev + 41\n[bt] (1) 1   libstdc++.6.dylib                   0x000000011b5e3300 _ZTVNSt7__cxx1115basic_stringbufIcSt11char_traitsIcESaIcEEE + 16\n'

In [ ]:
# train final model on the full data set
params = {
    "eta": 0.002,
    "max_depth": 3, 
    "objective": "survival:cox",
    "subsample": 0.5
}
model = xgboost.train(params, xgb_full, 5000, evals = [(xgb_full, "test")], verbose_eval=1000)

In [32]:
X,y = shap.datasets.nhanesi()

In [33]:
type(y)

numpy.ndarray

In [34]:
y

array([ 15.27465753,  11.58607306,   8.14908676, ..., -18.87716895,
       -19.68310502,  17.93858447])